In [30]:
%load_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader
from dataset import RecoveryRateDataset
import os
from scaling_utils import compute_mean_and_std, apply_std_scaling
from model import LSTMEstimator
from training_loop import model_train

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
root_dir = os.getcwd().split('Start_PyCharm')[0]
data_file_path = root_dir + 'Start_PyCharm/state_vector_dataframes_as_training_data/data/airport_state_df_w_features.pickle'


SEQUENCE_LENGTH = 5

features_to_drop = ['reg_type', 'reg_bool_type', 'reg_cause']
rr_dataset = RecoveryRateDataset(data_file_path, features_to_drop=features_to_drop, sequence_length=SEQUENCE_LENGTH,
                                 fill_with='backfill')


train_val_test_ratios = (0.6, 0.2, 0.2)
train, val, test = torch.utils.data.random_split(rr_dataset, lengths=[int(len(rr_dataset)*ratio)
                                                                      for ratio in train_val_test_ratios])

#Compute mean and std for standard scaling.
training_mean, training_std = compute_mean_and_std(train)

scaled_training_data = apply_std_scaling(train, training_mean, training_std)
scaled_val_data = apply_std_scaling(val, training_mean, training_std)
scaled_test_data = apply_std_scaling(test, training_mean, training_std)


#Training parameters.
BATCH_SIZE = 64
LR = 0.01
N_EPOCHS = 50

training_data_loader = DataLoader(scaled_training_data, batch_size=BATCH_SIZE, shuffle=True)
val_data_loader = DataLoader(scaled_val_data)
test_data_loader = DataLoader(scaled_test_data)

In [31]:
model = LSTMEstimator(len(rr_dataset.feature_names), initial_dense_layer_size=50, dense_parameter_multiplier=2,
                      dense_layer_count=3, lstm_layer_count=3, lstm_hidden_units=50, sequence_length=SEQUENCE_LENGTH)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
lr_scheduler = None # Optional
loss_func = torch.nn.MSELoss()

model_save_path='best_model'

train_history = []
val_history = []

model_train(model, optimizer, loss_func, N_EPOCHS, training_data_loader, val_data_loader, train_history, val_history, model_save_path)

100%|██████████| 1084/1084 [00:11<00:00, 90.98it/s]



 Starting epoch 0, loss: 0.86113457270379, val_loss: 0.01903850957751274 lr= 0.01

 Validation loss inf --> 0.7828313380863088. Saving best model.


100%|██████████| 1084/1084 [00:16<00:00, 65.24it/s]



 Starting epoch 1, loss: 0.8560404142165536, val_loss: 0.016205113381147385 lr= 0.01


100%|██████████| 1084/1084 [00:18<00:00, 57.77it/s]


...


 Starting epoch 25, loss: 1.1373353656587566, val_loss: 0.03674255311489105 lr= 0.01


100%|██████████| 1084/1084 [00:14<00:00, 74.41it/s]



 Starting epoch 26, loss: 1.1373854179168978, val_loss: 0.04330998659133911 lr= 0.01


100%|██████████| 1084/1084 [00:15<00:00, 70.92it/s]


KeyboardInterrupt: 